# Finetuning T5 for Size Selection

The purpose of this notebook is to document the process of finetuning Google's T5 model for translating from Literary Tibetan to English. This notebook relies on a dataset in the form of a pickled pandas dataframe which consists of a single column, 'translation'. Entries in that column should be a python dictionary of the structure: {'bo':'Tibetan text', 'en': 'English text'}.

In creating this notebook I drew on the following tutorial from HuggingFace: https://huggingface.co/learn/nlp-course/chapter7/4?fw=pt

In [ ]:
# only this line should be changed for testing different sizes
# must be one of ['small', 'base', large', '3b']
size = 'small'

## Load and Split the Data for Training

In [ ]:
from datasets import load_dataset
train_dataset = load_dataset('pandas', data_files='/home/j/Documents/Projects/MLotsawa/data/size-selection-data/train.p', streaming=True)
eval_dataset = load_dataset('pandas', data_files='/home/j/Documents/Projects/MLotsawa/data/size-selection-data/eval.p', streaming=True)

## Format and Tokenize the Data

This notebook uses Google's T5-small model and its associated tokenizer. This model gives really great results on translation tasks despite its small size. The data must be reformatted though to accomodate the model's expectations.

We will format each input as 'translate Tibetan to English: \<Tibetan text\>' with the English translation as the target.

Once the sentence pairs are formatted, we can tokenize them for processing by the model.

In [ ]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq

checkpoint = "google-t5/t5-{size}"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
source_lang = 'bo'
target_lang = 'en'
prefix = "translate Tibetan to English: "

def preprocess_function(examples):

    inputs = [prefix + example[source_lang] for example in examples['translation']]
    targets = [example[target_lang] for example in examples['translation']]
    
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)

    return model_inputs


In [ ]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

## Prepare Evaluation Metrics

We will be using the BLEU metric as implemented by SacreBLEU as our evaluation metric. BLEU (BiLingual Evaluation Understudy) is a standard (if not uncontroversial) metric in machine translation. BLEU gives each prediction a score between 0 and 1, where 0 means the model's predicted translation is nothing like the correct translation and 1 means the predicited translation is identical to the correct one. You can read more about the specifics here: https://en.wikipedia.org/wiki/BLEU 

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

## Load and Train the Model

Finally, we train the model. You can uncomment the line 'model.to("cuda:0")' if are working on a machine that has a CUDA compatible GPU. The training arguments below are taken from the HuggingFace tutorial cited in above. 

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback

early_stop = EarlyStoppingCallback()

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model.to("cuda:0")

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="../../models/size-selection/{size}/",
    learning_rate=2e-5,
    auto_find_batch_size=True,
    weight_decay=0.01,
    save_total_limit=5,
    num_train_epochs=30,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    save_steps=25000,
    load_best_model_at_end=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callback=[early_stop]
)

trainer.train()